In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F
from tqdm import tqdm
from torch.utils import data
import wandb

NUM_SAVE=50
net_list="in->256->64"

class MLP(nn.Module):
    def __init__(self,in_features):
        super().__init__()
        self.layer1=nn.Linear(in_features,256)
        self.layer2=nn.Linear(256,64)
        self.out=nn.Linear(64,1)
    def forward(self,X):
        X=F.relu(self.layer1(X))
        X=F.relu(self.layer2(X))
        return self.out(X)

In [ ]:
device=torch.device("cuda:0" if torch.cuda.is_available else "cpu")
test_data=pd.read_csv('./house_price/test.csv')
train_data=pd.read_csv('./house_price/train.csv')
print("train_data and test_data shape",train_data.shape,test_data.shape)
# train_data and test_data shape (1460, 81) (1459, 80)
type(train_data)
# pandas.core.frame.DataFrame

# 看一下前4个特征和最后两个特征，以及相应标签
print(train_data.iloc[0:4,[0,1,2,3,-3,-2,-1]])
# 去掉第一个id标签以及训练集的label，去掉id是因为它不携带任何用于预测的信息，然后把训练集和测试集合并
all_features=pd.concat((train_data.iloc[:,1:-1],test_data.iloc[:,1:]))
print(all_features.shape)


## 数据预处理

首先将所有确实的值替换为相应特征的平均值；
为了将所有特征放在一个共同的尺度上，我们通过将特征重新缩放到零均值和单位方差来标准化数据；
标准化数据的两个原因：
1. 方便优化
2. 因为不知道那些特征是相关的，所以不想让惩罚分配给一个特征的系数比分配给其他任何特征的系数更大。



In [ ]:
# 若无法获得测试数据，则可根据训练数据计算均值和标准差
numeric_features=all_features.dtypes[all_features.dtypes !='object'].index  # 代码理解：https://www.cnblogs.com/hahaah/p/15067930.html
print(numeric_features)
all_features[numeric_features]=all_features[numeric_features].apply(lambda x:(x-x.mean())/(x.std()))
# 在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为0
all_features[numeric_features]=all_features[numeric_features].fillna(0)


### 接下来处理离散值
用独热编码来替换它们，例如，“MSZoning”包含值“RL”和“Rm”。 我们将创建两个新的指示器特征“MSZoning_RL”和“MSZoning_RM”，其值为0或1。 根据独热编码，如果“MSZoning”的原始值为“RL”， 则：“MSZoning_RL”为1，“MSZoning_RM”为0。 pandas软件包会自动为我们实现这一点。

In [ ]:
# "Dummy_Na=True"将"na"(缺失值)视为有效的特征值，并为其创建指示符特征
all_features=pd.get_dummies(all_features,dummy_na=True)
all_features.shape # 特征总数零从79增加到了331个，


In [ ]:
# 最后，通过values属性，我们可以从pandas格式中提取NumPy格式，并将其转换为张量表示用于训练
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float32)
test_features = torch.tensor(all_features[n_train:].values, dtype=torch.float32)
train_labels = torch.tensor(train_data.SalePrice.values.reshape(-1, 1), dtype=torch.float32)

## 训练


In [33]:
criterion=nn.MSELoss()
in_features=train_features.shape[1]
net=MLP(in_features).to(device)

def load_array(data_arrays,batch_size,is_train=True): #@save
    """Construct a Pytorch data iterator."""
    dataset=data.TensorDataset(*data_arrays)  # 将特征和lables组合在一起
    return data.DataLoader(dataset,batch_size,shuffle=is_train)


# 用价格预测的对数来衡量差异
def log_mse(net,features,labels):
    # 为了在取对数时进一步稳定该值，将小于1的值设置为1
    clipped_preds=torch.clamp(net(features),1,float('inf'))
    rmse=torch.sqrt(criterion(torch.log(clipped_preds),torch.log(labels)))
    return rmse.item()

def train(net, train_features,train_labels,test_features,test_labels, num_epochs,learning_rate, weight_decay, batch_size):
    wandb.watch(net)
    train_ls, test_ls=[], []
    train_iter=load_array((train_features,train_labels),batch_size)
    # 这里使用的是Adam优化算法
    optimizer=torch.optim.Adam(net.parameters(),lr=learning_rate,weight_decay=weight_decay)
    for epoch in tqdm(range(num_epochs)): # tqdm进度条
        for X,y in train_iter:
            X,y=X.to(device),y.to(device)
            optimizer.zero_grad()
            outputs=net(X)
            loss=criterion(outputs,y)
            loss.backward()
            optimizer.step()
            
        record_loss=log_mse(net.to('cpu'),train_features,train_labels)
        wandb.log({'loss':record_loss,'epoch':epoch})
        train_ls.append(record_loss)
        
        if (epoch%NUM_SAVE==0 and epoch!=0) or(epoch==num_epochs-1):
            torch.save(net.state_dict(),'checkpoint_'+str(epoch))
            print('save checkpoints on:',epoch,'rmse loss value is:',record_loss)
        del X,y
        net.to(device)
    wandb.finish()
    return train_ls,test_ls

        
k, num_epochs, lr, weight_decay, batch_size = 5, 2000, 0.005, 0.05, 256
wandb.init(project="kaggle_predict",
           config={ "learning_rate": lr,
                    "weight_decay": weight_decay,
                    "batch_size": batch_size,
                    "total_run": num_epochs,
                    "network": net_list}
          )
print("network:",net)   
    

network: MLP(
  (layer1): Linear(in_features=331, out_features=256, bias=True)
  (layer2): Linear(in_features=256, out_features=64, bias=True)
  (out): Linear(in_features=64, out_features=1, bias=True)
)


In [34]:
train_ls, valid_ls = train(net, train_features,train_labels,None,None, num_epochs, lr, weight_decay, batch_size)

# 使用现有训练好的net
net.to('cpu')
# 将网络应用于测试集。
preds = net(test_features).detach().numpy() # 用detach()切断反向传播

# 将其重新格式化以导出到Kaggle
test_data['SalePrice'] = pd.Series(preds.reshape(1, -1)[0])
submission = pd.concat([test_data['Id'], test_data['SalePrice']], axis=1)
submission.to_csv('submission.csv', index=False)

  3%|██▏                                                                             | 54/2000 [00:02<01:23, 23.27it/s]

save checkpoints on: 50 rmse loss value is: 0.14847400784492493


  5%|████                                                                           | 102/2000 [00:04<01:28, 21.43it/s]

save checkpoints on: 100 rmse loss value is: 0.1325608193874359


  8%|██████                                                                         | 155/2000 [00:07<01:22, 22.46it/s]

...

save checkpoints on: 1950 rmse loss value is: 0.04175592213869095


100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [02:01<00:00, 16.41it/s]


save checkpoints on: 1999 rmse loss value is: 0.039992429316043854


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,1999
loss,0.03999


KeyError: 'SalePrice'

In [36]:
# 将其重新格式化以导出到Kaggle
test_data['SalePrice'] = pd.Series(preds.reshape(1, -1)[0])
submission = pd.concat([test_data['Id'], test_data['SalePrice']], axis=1)
submission.to_csv('submission.csv', index=False)